In [2]:
import matplotlib.pyplot as plt
import numpy as np

from ESN import EchoStateNetwork
from ESN import MackeyGlassGenerator
from ESN import GinfActivator

ginf_activator = GinfActivator(V_min=-2, V_max=2, resolution=200, offset=True)

def mse(test, predictions):
    return np.mean((predictions[100:] - test[100:]) ** 2)

In [8]:
import optuna

def relu(x):
    return np.clip(x, 0, 2)

def leaky_relu(x, alpha=0.01):
    return np.where(x > 0, x, alpha * x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softsign(x):
    return x / (1 + np.abs(x))

activation_functions = {
    "tanh": np.tanh,
    "clip": relu,
    "leaky_relu": leaky_relu,
    "sigmoid": sigmoid,
    "softsign": softsign,
    "ginf": ginf_activator.activate
}

In [9]:
def objective(trial):
    tau = np.random.randint(10, 30)
    trial.set_user_attr("tau", tau)

    mg_series = MackeyGlassGenerator(tau=tau, n=10000, n_samples=5000)
    mg_series = 2 * (mg_series - mg_series.min()) / (mg_series.max() - mg_series.min()) - 1

    inputs = mg_series[:-1].reshape(-1, 1)
    targets = mg_series[1:].reshape(-1, 1)

    train_len = 4000
    test_len = 1000
    train_inputs = inputs[:train_len]
    train_targets = targets[:train_len]
    test_inputs = inputs[train_len : train_len + test_len]
    test_targets = targets[train_len : train_len + test_len]

    reservoir_size = trial.suggest_int("reservoir_size", 1, 400, log=True)
    leaking_rate = trial.suggest_float("leaking_rate", 0.01, 1.0)
    step_size = trial.suggest_float("step_size", 0.0001, 1, log=True)
    time_scale = trial.suggest_float("time_scale", 0.1, 5.0)
    spectral_radius = trial.suggest_float("spectral_radius", 0.1, 1)
    sparsity = trial.suggest_float("sparsity", 0.01, 0.99)
    input_scaling = trial.suggest_float("input_scaling", 0.1, 10.0, log=True)
    regularization = trial.suggest_float("regularization", 1e-6, 1e-2, log=True)

    activation_name = trial.suggest_categorical("activation", list(activation_functions.keys()))
    trial.set_user_attr("Activation name", activation_name)
    activation_func = activation_functions[activation_name]

    # Enforce the constraint before constructing the ESN
    if leaking_rate * (step_size / time_scale) > 1:
        raise optuna.exceptions.TrialPruned("Leaking rate * step_size/time_scale > 1")

    esn = EchoStateNetwork(
        input_dim=1,
        reservoir_size=reservoir_size,
        output_dim=1,
        leaking_rate=leaking_rate,
        step_size=step_size,
        time_scale=time_scale,
        spectral_radius=spectral_radius,
        sparsity=sparsity,
        input_scaling=input_scaling,
        regularization=regularization,
        activation=activation_func,
        guarantee_ESP=True,
    )
    esn.fit(train_inputs, train_targets, washout=200)

    predictions = esn.predict(test_inputs)
    loss = mse(test_targets, predictions)
    return min(loss, 1)

In [10]:
try:
    study_name="ESN optimization MG 🔥"
    storage="sqlite:///optuna_esn_MG.db"

    optuna.delete_study(study_name=study_name, storage=storage)
except Exception:
    print("No database available")

In [ ]:
study = optuna.create_study(
    direction="minimize",
    study_name=study_name,
    storage=storage,
    load_if_exists=True,
)
study.optimize(objective, n_trials=600, n_jobs=-1)

best_params = study.best_params
print("Best Hyperparameters:", best_params)

In [ ]:
!optuna-dashboard sqlite:///optuna_esn_MG.db